In [1]:
!pip install cherche

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.4/385.4 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 86.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [2]:
from cherche import retrieve

In [3]:

import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
caminho_query = "/content/gdrive/MyDrive/Colab Notebooks/dados-conle-anonimizado-assunto-notnull (1).csv"

Mounted at /content/gdrive


# Base de dados câmara dos deputados

In [5]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/proposicao-tema-completo.csv")

In [6]:
df_assunto= pd.read_csv(caminho_query, encoding='utf-8', delimiter=";")

In [7]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [8]:
def verificar(y,top_n):
    for d in top_n:
      if y == df.loc[d["id"],"txtNome"]:
        return 1
    return 0

In [9]:
def avaliacaoRecall(isPreprocess):

    quant_encontrado=0
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = retriever(tokenized_query3)
        #top_n = [df.loc[d["id"],"name"] for d in top_n_stem_l]

        
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    print("R@20: "+str(recall))


In [10]:
def tokenizer1(doc):
    return doc.split("\n")

# Word n-gram + pré processamento básico

## 15- Letra mínuscula + remoção de pontuação, acentuação e stopword + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]



In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.4847457627118644


In [ ]:
documentos = None
retriever =None

## 16- Letra mínuscula + remoção de pontuação, acentuação e stopword + trigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + " " + w[1] + " " + w[2]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.376271186440678


In [ ]:
documentos = None
retriever =None

## 17- Letra mínuscula + remoção de pontuação, acentuação e stopword + unigram + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)


In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.5491525423728814


# Word n-gram

In [15]:
def tokenizer1(doc):
    return doc.split("\n")

## 12- Bigram

In [ ]:
def preprocess(txt):

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    ngram = []
    
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.4576271186440678


In [ ]:
documentos = None
retriever =None

## 13- Trigram

In [12]:
def preprocess(txt):

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    ngram = []
    
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + " " + w[1] + " " + w[2]
        ngram.append(string)
    
    return "\n".join(ngram)

In [13]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





KeyboardInterrupt: ignored

In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.36271186440677966


In [ ]:
documentos = None
retriever =None

## 14- Unigram + Bigram

In [11]:
def preprocess(txt):
    
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [12]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [13]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [14]:
avaliacaoRecall(True)

R@20: 0.5016949152542373


In [ ]:
documentos = None
retriever =None